In [4]:
import pandas as pd
import os
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt


from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Data Science Project/data"
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_bids = pd.read_csv("bids.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Data Science Project/data


In [0]:
##data_bids = data_bids.replace({' ': ''}, regex = True)

In [7]:
data_bids 

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,9709222052631578,ru,140.204.227.63,cghhmomsaxi6pug
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,9709222052631578,pl,24.232.159.118,wgggpdg2gx5pesn
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,9709222052631578,za,80.237.28.246,5xgysg14grlersa
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,9709222052631578,my,91.162.27.152,bhtrek44bzi2wfl


In [0]:
data_bids_sorted = data_bids.sort_values(by=['bidder_id', 'time'], ascending = [True, True]) ##sorting the data by the bidder_id and time 

In [0]:
bids = pd.DataFrame(data = data_bids_sorted['bidder_id'].unique(), columns = ['bidder_id'], index = data_bids_sorted['bidder_id'].unique())
## new dataframe for all info of each bidder_id 

In [0]:
counts = data_bids_sorted["bidder_id"].value_counts()
bids['auction_count_num'] = counts

In [0]:
timediff = data_bids_sorted.groupby('bidder_id')['time'].diff()
timediff_str = timediff.astype(str).fillna('')
data_bids_sorted['timediff_num'] = timediff
data_bids_sorted['timediff'] = timediff_str

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,timediff_num,timediff
7179832,7179832,001068c415025a009fee375a12cff4fcnht8y,4ifac,jewelry,phone561,9706345052631578,bn,139.226.147.115,vasstdc27m7nks3,NaN,nan
1281292,1281292,002d229ffb247009810828f648afc2ef593rb,2tdw2,mobile,phone640,9766744105263157,sg,37.40.254.131,vasstdc27m7nks3,NaN,nan
1281311,1281311,002d229ffb247009810828f648afc2ef593rb,2tdw2,mobile,phone219,9766744210526315,sg,37.40.254.131,vasstdc27m7nks3,1.052632e+08,105263158.0
6805028,6805028,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,mobile,phone313,9704553947368421,ir,21.67.17.162,vnw40k8zzokijsv,NaN,nan
3967330,3967330,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,obbny,mobile,phone420,9640018631578947,id,44.241.8.179,sj4jidex850loas,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...
2336021,2336021,ffd62646d600b759a985d45918bd6f0431vmz,dwy5m,mobile,phone293,9772802947368421,id,214.13.99.0,zkb9nv4gtdm2vrw,5.631579e+09,5631578948.0
2337626,2337626,ffd62646d600b759a985d45918bd6f0431vmz,kh2ef,mobile,phone195,9772811421052631,id,144.253.142.62,vasstdc27m7nks3,8.473684e+09,8473684210.0
2344645,2344645,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,mobile,phone153,9772850052631578,id,166.67.25.123,l69hkdm98yfwmu0,3.863158e+10,38631578947.0
2350507,2350507,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,mobile,phone4,9772881631578947,id,214.13.99.0,vasstdc27m7nks3,3.157895e+10,31578947369.0
